#Statistical Analysis

In [ ]:
#Setting up goggle drive as main directory
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Downloading the dataset
!git clone https://github.com/epic-kitchens/epic-kitchens-100-hand-object-bboxes

Cloning into 'epic-kitchens-100-hand-object-bboxes'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 194 (delta 85), reused 154 (delta 54), pack-reused 0
Receiving objects: 100% (194/194), 946.33 KiB | 10.75 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
#Installing the epic-kitchens module
!pip install epic-kitchens

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached epic-kitchens-1.7.1.tar.gz (53 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.7.0.tar.gz (51 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.6.3.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.6.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.6.0.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.5.0.tar.gz (31 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.4.0.tar.gz (29 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.3.0.tar.gz (28 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.2.1.tar.gz (26 kB)
  Preparing metadata (setup.py) ... done
  Using cached epic-kitchens-1.2.0.tar.gz (27 kB)
  Preparing metadata (setup.py) 

In [ ]:
%cd /content/epic-kitchens-100-hand-object-bboxes

/content/epic-kitchens-100-hand-object-bboxes


In [ ]:
#installing the automatic annotations 
!python3 setup.py install

running install
running bdist_egg
running egg_info
writing src/public_lib/epic_hand_object_detections.egg-info/PKG-INFO
writing dependency_links to src/public_lib/epic_hand_object_detections.egg-info/dependency_links.txt
writing requirements to src/public_lib/epic_hand_object_detections.egg-info/requires.txt
writing top-level names to src/public_lib/epic_hand_object_detections.egg-info/top_level.txt
writing manifest file 'src/public_lib/epic_hand_object_detections.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/epic_kitchens
copying build/lib/epic_kitchens/__init__.py -> build/bdist.linux-x86_64/egg/epic_kitchens
creating build/bdist.linux-x86_64/egg/epic_kitchens/hoa
copying build/lib/epic_kitchens/hoa/types_pb2.py -> build/bdist.linux-x86_64/egg/epic_kitchens/hoa
copying build/lib/epic_kitchens/hoa/visualisation.py -> build/bdist.linux-x86_64/

In [ ]:
%cd src/

/content/epic-kitchens-100-hand-object-bboxes/src


In [ ]:
%cd public_lib/

/content/epic-kitchens-100-hand-object-bboxes/src/public_lib


In [ ]:
#Importing the required libraries to load the automatic detections
from typing import Union, List
from pathlib import Path
import re
from ipywidgets import interact, IntSlider, Layout

from epic_kitchens.hoa import load_detections, DetectionRenderer
import PIL.Image

In [ ]:
#Accessing the folder where all the frame annotations are
#glob turns all the content of the folders into a list 
#Counting the total number of annotation files

import glob

pkl_folder = sorted(glob.glob(r'/content/gdrive/MyDrive/hand-objects/detections_root/P37/*'))

print("Total number of files: " + str(len(pkl_folder)))

Total number of files: 3


In [ ]:
#creating folder to save extracted data into csvs
import os

os.mkdir('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant')

In [ ]:
%cd /content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant

/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant


In [ ]:
#Extracts data from automatic annotations and saves it into a table format csv file

frame_height = 256 
frame_width = 456

handz = []
hands = []

for detec in pkl_folder:
  video_find = str(detec)
  detections_root = Path(video_find[0:56])
  video_id = video_find[57:]
  detections = load_detections(detections_root / (video_id))
  
  for detection in detections:
      detection.scale(height_factor=frame_height)
      fr = detection.frame_number
      for hand in detection.hands:
        hands.append(hand)
        hand_y = hand.bbox.center[1]
        hand_x = ((hand.bbox.left * frame_width) + ((hand.bbox.width * frame_width) / 2)) 
        hand_state = hand.state.name

        if hand.side.value == 0:
          z = video_id + "," + str(fr) + "," + "left" + "," + str(hand_y) + "," + str(round(hand_x, 2)) + "," + str(hand_state)
          handz.append(z)
        if hand.side.value == 1:
          z1 = video_id + "," + str(fr) + "," + "right" + "," + str(hand_y) + "," + str(round(hand_x, 2)) + "," + str(hand_state)
          handz.append(z1)
  x = "Participant, Frame, Hand, Centre_Y, Centre_X, Contact"
  handz.insert(0, x)

  textfile = open(video_id[:-4] + ".csv", "w")
  for element in handz:
    textfile.write(element + "\n")
  handz.clear()
  hands.clear()

#accessing the specific folder where the csv's are

import glob

csvs_folder = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/*.csv'))

# Concatenate all data into one DataFrame

import pandas as pd

dfs = []

for filename in csvs_folder:

    dfs.append(pd.read_csv(filename))

big_frame = pd.concat(dfs, ignore_index=True)

big_frame.to_csv('P37.csv')

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/P37.csv')
df

,Unnamed: 0,Participant,Frame,Hand,Centre_Y,Centre_X,Contact
0,0,P37_101.pkl,1,left,194.0,67.5,NO_CONTACT
1,1,P37_101.pkl,1,right,194.5,401.5,PORTABLE_OBJECT
2,2,P37_101.pkl,2,left,197.0,64.0,NO_CONTACT
3,3,P37_101.pkl,2,right,193.0,402.0,PORTABLE_OBJECT
4,4,P37_101.pkl,3,left,199.5,62.0,NO_CONTACT
...,...,...,...,...,...,...,...
253027,253027,P37_103.pkl,19318,right,136.5,365.5,STATIONARY_OBJECT
253028,253028,P37_103.pkl,19319,right,136.0,365.0,STATIONARY_OBJECT
253029,253029,P37_103.pkl,19320,right,137.0,365.0,STATIONARY_OBJECT
253030,253030,P37_103.pkl,19321,right,136.0,365.0,STATIONARY_OBJECT


In [ ]:
#Turning Centre_Y column into a list
#Centre_Y was calculated based on image classification so 0 is the top left corner
#Centre_Y therefore needs to be converted to cartesian so 0 is the bottom left corner

Centre_Y_list = df[" Centre_Y"].tolist()
Centre_Y_list[:10] #printing first 10 values to check they make sense

[194.0, 194.5, 197.0, 193.0, 199.5, 196.5, 201.5, 199.5, 204.5, 203.0]

In [ ]:
#Converting Y_centre to cartesian 
#Add values to new column
new_Centre_Y_list = []

for c in Centre_Y_list:
  n = 256 - c
  new_Centre_Y_list.append(n)

new_Centre_Y_list[:10]

[62.0, 61.5, 59.0, 63.0, 56.5, 59.5, 54.5, 56.5, 51.5, 53.0]

In [ ]:
#Adding new column to dataframe
df.insert(loc = 6,
          column = 'cart_Centre_Y',
          value = new_Centre_Y_list)

# show the dataframe
df

,Unnamed: 0,Participant,Frame,Hand,Centre_Y,Centre_X,cart_Centre_Y,Contact
0,0,P37_101.pkl,1,left,194.0,67.5,62.0,NO_CONTACT
1,1,P37_101.pkl,1,right,194.5,401.5,61.5,PORTABLE_OBJECT
2,2,P37_101.pkl,2,left,197.0,64.0,59.0,NO_CONTACT
3,3,P37_101.pkl,2,right,193.0,402.0,63.0,PORTABLE_OBJECT
4,4,P37_101.pkl,3,left,199.5,62.0,56.5,NO_CONTACT
...,...,...,...,...,...,...,...,...
253027,253027,P37_103.pkl,19318,right,136.5,365.5,119.5,STATIONARY_OBJECT
253028,253028,P37_103.pkl,19319,right,136.0,365.0,120.0,STATIONARY_OBJECT
253029,253029,P37_103.pkl,19320,right,137.0,365.0,119.0,STATIONARY_OBJECT
253030,253030,P37_103.pkl,19321,right,136.0,365.0,120.0,STATIONARY_OBJECT


In [ ]:
#Removing Centre_Y column
df.drop(big_frame.columns[3], axis=1, inplace=True)

In [ ]:
df

,Unnamed: 0,Participant,Frame,Hand,Centre_X,cart_Centre_Y,Contact
0,0,P37_101.pkl,1,left,67.5,62.0,NO_CONTACT
1,1,P37_101.pkl,1,right,401.5,61.5,PORTABLE_OBJECT
2,2,P37_101.pkl,2,left,64.0,59.0,NO_CONTACT
3,3,P37_101.pkl,2,right,402.0,63.0,PORTABLE_OBJECT
4,4,P37_101.pkl,3,left,62.0,56.5,NO_CONTACT
...,...,...,...,...,...,...,...
253027,253027,P37_103.pkl,19318,right,365.5,119.5,STATIONARY_OBJECT
253028,253028,P37_103.pkl,19319,right,365.0,120.0,STATIONARY_OBJECT
253029,253029,P37_103.pkl,19320,right,365.0,119.0,STATIONARY_OBJECT
253030,253030,P37_103.pkl,19321,right,365.0,120.0,STATIONARY_OBJECT


In [ ]:
df.to_csv('P37.csv', index = False)

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/P37.csv')
df

,Unnamed: 0,Participant,Frame,Hand,Centre_X,cart_Centre_Y,Contact
0,0,P37_101.pkl,1,left,67.5,62.0,NO_CONTACT
1,1,P37_101.pkl,1,right,401.5,61.5,PORTABLE_OBJECT
2,2,P37_101.pkl,2,left,64.0,59.0,NO_CONTACT
3,3,P37_101.pkl,2,right,402.0,63.0,PORTABLE_OBJECT
4,4,P37_101.pkl,3,left,62.0,56.5,NO_CONTACT
...,...,...,...,...,...,...,...
253027,253027,P37_103.pkl,19318,right,365.5,119.5,STATIONARY_OBJECT
253028,253028,P37_103.pkl,19319,right,365.0,120.0,STATIONARY_OBJECT
253029,253029,P37_103.pkl,19320,right,365.0,119.0,STATIONARY_OBJECT
253030,253030,P37_103.pkl,19321,right,365.0,120.0,STATIONARY_OBJECT


In [ ]:
#for loop to pull data from participant csv and group info into the columns mentioned below - hand position in space

hands_csv = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228

values = str(len(df[' Hand'])) + "," + str(len(df[hands_left])) + "," + str(len(df[hands_left & hands_upper])) + "," + str(len(df[hands_left & hands_lower])) + "," + str(len(df[hands_left & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_Semi_Right])) + "," + str(len(df[hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_right])) + "," + str(len(df[hands_right & hands_upper])) + "," + str(len(df[hands_right & hands_lower]))+ ","  + str(len(df[hands_right & hands_Semi_Left])) + "," +str(len(df[hands_right & hands_Semi_Right])) + "," + str(len(df[hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_right & hands_lower & hands_Semi_Right])) 
  
hands_csv.append(values)
                                 
x = "hands_total, hands_left, left_upper, left_lower, left_ipsi, left_contra, left_ULquad, left_LLquad, leftURquad, leftLRquad, hands_right, right_upper, right_lower, right_contra, right_ipsi, right_ULquad, right_LLquad, right_URquad, right_LRquad" 
  
hands_csv.insert(0, x)

textfile = open("P37_Hands_overall.csv", "w")
for element in hands_csv:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
hands = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/P37_Hands_overall.csv')
hands

,hands_total,hands_left,left_upper,left_lower,left_ipsi,left_contra,left_ULquad,left_LLquad,leftURquad,leftLRquad,hands_right,right_upper,right_lower,right_contra,right_ipsi,right_ULquad,right_LLquad,right_URquad,right_LRquad
0,253032,121700,35413,86287,83037,38663,24560,58477,10853,27810,131332,33657,97675,4166,127166,786,3380,32871,94295


In [ ]:
#for loop to pull data from participant csv and group info into visual field areas and no object interaction

df_NC = df

noContact_csv = []

vid_id = str(df_NC['Participant'])
vid_name = vid_id[:-4]
hands_left = df_NC[' Hand'] == 'left'
hands_right = df_NC[' Hand'] == 'right'
hands_lower = df_NC['cart_Centre_Y'] < 128
hands_upper = df_NC['cart_Centre_Y'] >= 128
hands_Semi_Right = df_NC[' Centre_X'] > 228
hands_Semi_Left = df_NC[' Centre_X'] <= 228
hands_noContact = df_NC[' Contact'] == 'NO_CONTACT'
hands_Portable = df_NC[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df_NC[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df_NC[' Contact'] == 'SELF_CONTACT'
  
values_NC = str(len(df_NC[' Hand'])) + "," + str(len(df_NC[hands_noContact])) + "," + str(len(df_NC[hands_noContact & hands_left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df_NC[hands_noContact & hands_right & hands_lower & hands_Semi_Right])) 
noContact_csv.append(values_NC)
                                      
x = "hands_total, noContact_total, noContact_left, noContact_left_upper, noContact_left_lower, noContact_left_ipsi, noContact_left_contra, noContact_left_ULquad, noContact_left_LLquad, noContact_left_URquad, noContact_left_LRquad, noContact_hands_right, noContact_right_upper, noContact_right_lower, noContact_right_contra, noContact_right_ipsi, noContact_right_ULquad, noContact_right_LLquad, noContact_right_URquad, noContact_right_LRquad"
noContact_csv.insert(0, x)

textfile_NC = open("P37_Hands_NoContact.csv", "w")
for element in noContact_csv:
    textfile_NC.write(element + "\n")
textfile_NC.close()

In [ ]:
df_NC_crushed = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/P37_Hands_NoContact.csv')
df_NC_crushed

,hands_total,noContact_total,noContact_left,noContact_left_upper,noContact_left_lower,noContact_left_ipsi,noContact_left_contra,noContact_left_ULquad,noContact_left_LLquad,noContact_left_URquad,noContact_left_LRquad,noContact_hands_right,noContact_right_upper,noContact_right_lower,noContact_right_contra,noContact_right_ipsi,noContact_right_ULquad,noContact_right_LLquad,noContact_right_URquad,noContact_right_LRquad
0,253032,19947,10889,1749,9140,8627,2262,1367,7260,382,1880,9058,2339,6719,509,8549,31,478,2308,6241


In [ ]:
#for loop to pull data from participant csv and group info into visual field areas and portable object interaction

hands_csv2 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Portable])) + "," + str(len(df[hands_Portable & hands_left])) + "," + str(len(df[hands_Portable & hands_left & hands_upper])) + "," + str(len(df[hands_Portable & hands_left & hands_lower])) + "," + str(len(df[hands_Portable & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right])) + "," + str(len(df[hands_Portable & hands_right & hands_upper])) + "," + str(len(df[hands_Portable & hands_right & hands_lower])) + "," + str(len(df[hands_Portable & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Portable & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Portable & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv2.append(values1)
                                      
x = "hands_total, portable_total, portable_left, portable_left_upper, portable_left_lower, portable_left_ipsi, portable_left_contra, portable_left_ULquad, portable_left_LLquad, portable_left_URquad, portable_left_LRquad, portable_hands_right, portablet_right_upper, portable_right_lower, portable_right_contra, portable_right_ipsi, portable_right_ULquad, portable_right_LLquad, portable_right_URquad, portable_right_LRquad"
hands_csv2.insert(0, x)

textfile = open("P37_Hands_Portable.csv", "w")
for element in hands_csv2:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
port_df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/P37_Hands_Portable.csv')
port_df

,hands_total,portable_total,portable_left,portable_left_upper,portable_left_lower,portable_left_ipsi,portable_left_contra,portable_left_ULquad,portable_left_LLquad,portable_left_URquad,portable_left_LRquad,portable_hands_right,portablet_right_upper,portable_right_lower,portable_right_contra,portable_right_ipsi,portable_right_ULquad,portable_right_LLquad,portable_right_URquad,portable_right_LRquad
0,253032,224964,107837,33083,74754,72779,35058,22843,49936,10240,24818,117127,29961,87166,3544,113583,745,2799,29216,84367


In [ ]:
#for loop to pull data from participant csv and group info into visual field areas and stationary object interaction

hands_csv3 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Stationary])) + "," + str(len(df[hands_Stationary & hands_left])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower])) + "," + str(len(df[hands_Stationary & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower])) + "," + str(len(df[hands_Stationary & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Stationary & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Stationary & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv3.append(values1)
                                      
x = "hands_total, Stationary_total, Stationary_left, Stationary_left_upper, Stationary_left_lower, Stationary_left_ipsi, Stationary_left_contra, Stationary_left_ULquad, Stationary_left_LLquad, Stationary_left_URquad, Stationary_left_LRquad, Stationary_hands_right, Stationary_right_upper, Stationary_right_lower, Stationary_right_contra, Stationary_right_ipsi, Stationary_right_ULquad, Stationary_right_LLquad, Stationary_right_URquad, Stationary_right_LRquad"
hands_csv3.insert(0, x)

textfile = open("P37_Hands_Stationary.csv", "w")
for element in hands_csv3:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
stat_df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/P37_Hands_Stationary.csv')
stat_df

,hands_total,Stationary_total,Stationary_left,Stationary_left_upper,Stationary_left_lower,Stationary_left_ipsi,Stationary_left_contra,Stationary_left_ULquad,Stationary_left_LLquad,Stationary_left_URquad,Stationary_left_LRquad,Stationary_hands_right,Stationary_right_upper,Stationary_right_lower,Stationary_right_contra,Stationary_right_ipsi,Stationary_right_ULquad,Stationary_right_LLquad,Stationary_right_URquad,Stationary_right_LRquad
0,253032,3987,1176,268,908,1031,145,230,801,38,107,2811,603,2208,15,2796,2,13,601,2195


In [ ]:
#for loop to pull data from participant csv and group info into visual field areas and contra-lateral hand interaction

hands_csv4 = []

vid_id = str(df['Participant'])
vid_name = vid_id[:-4]
hands_left = df[' Hand'] == 'left'
hands_right = df[' Hand'] == 'right'
hands_lower = df['cart_Centre_Y'] < 128
hands_upper = df['cart_Centre_Y'] >= 128
hands_Semi_Right = df[' Centre_X'] > 228
hands_Semi_Left = df[' Centre_X'] <= 228
hands_noContact = df[' Contact'] == 'NO_CONTACT'
hands_Portable = df[' Contact'] == 'PORTABLE_OBJECT'
hands_Stationary  = df[' Contact'] == 'STATIONARY_OBJECT'
hands_Self = df[' Contact'] == 'SELF_CONTACT'
  
values1 = str(len(df[' Hand'])) + "," + str(len(df[hands_Self])) + "," + str(len(df[hands_Self & hands_left])) + "," + str(len(df[hands_Self & hands_left & hands_upper])) + "," + str(len(df[hands_Self & hands_left & hands_lower])) + "," + str(len(df[hands_Self & hands_left & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_left & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_left & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_left & hands_lower & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right])) + "," + str(len(df[hands_Self & hands_right & hands_upper])) + "," + str(len(df[hands_Self & hands_right & hands_lower])) + "," + str(len(df[hands_Self & hands_right & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right & hands_upper & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_lower & hands_Semi_Left])) + "," + str(len(df[hands_Self & hands_right & hands_upper & hands_Semi_Right])) + "," + str(len(df[hands_Self & hands_right & hands_lower & hands_Semi_Right])) 
hands_csv4.append(values1)
                                      
x = "hands_total, hands_Self_total, hands_Self_left, hands_Self_left_upper, hands_Self_left_lower, hands_Self_left_ipsi, hands_Self_left_contra, hands_Self_left_ULquad, hands_Self_left_LLquad, hands_Self_left_URquad, hands_Self_left_LRquad, hands_Self_hands_right, hands_Self_right_upper, hands_Self_right_lower, hands_Self_right_contra, hands_Self_right_ipsi, hands_Self_right_ULquad, hands_Self_right_LLquad, hands_Self_right_URquad, hands_Self_right_LRquad"
hands_csv4.insert(0, x)

textfile = open("P37_Hands_Self.csv", "w")
for element in hands_csv4:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
self_df = pd.read_csv('/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/P37_Hands_Self.csv')
self_df

,hands_total,hands_Self_total,hands_Self_left,hands_Self_left_upper,hands_Self_left_lower,hands_Self_left_ipsi,hands_Self_left_contra,hands_Self_left_ULquad,hands_Self_left_LLquad,hands_Self_left_URquad,hands_Self_left_LRquad,hands_Self_hands_right,hands_Self_right_upper,hands_Self_right_lower,hands_Self_right_contra,hands_Self_right_ipsi,hands_Self_right_ULquad,hands_Self_right_LLquad,hands_Self_right_URquad,hands_Self_right_LRquad
0,253032,4134,1798,313,1485,600,1198,120,480,193,1005,2336,754,1582,98,2238,8,90,746,1492


In [ ]:
!pip install pingouin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 KB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 86.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pingouin: filename=pingouin-0.5.3-py3-none-any.whl size=198588 sha256=6b177557e0ae51690f5d02cf9958d74deb77d9ef2a72db5eaf3b7e50526c6621
  Stored in directory: /root/.cache/pip/wheels/60/1e/21/710be95d0c93ea9ac3c0e6bacd8cce492ad5dd26bf3aedc4bf
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7047 sha256=f7ee74df51f28d62a0ef271eff1537d0f6fd099045fff4348c2246d21315c0ce
  Stored in directory: /root/.cache/pip/wheels/6a/33/c4/0ef84d7f5568c2823e3d63a6e08988852fb9e4bc822034870a
Successfully built pingouin littleuti

In [ ]:
!pip install researchpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Statistical differences between hemi scenes

##Upper vs lower hemi scenes

In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

overall_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/Hands_overall/*.csv'))

print("Total number of files: " + str(len(overall_counts)))

Total number of files: 37


In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

upper = []
lower = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = ((values[2] + values[11]) / values[0]) * 100 #hands in the upper scene
  upper.append(a)
  b = ((values[3] + values[12]) / values[0]) * 100 #hands in the lower scene
  lower.append(b)

In [ ]:
df_j = pd.DataFrame(list(zip(upper, lower)), columns = ['upper', 'lower'])

In [ ]:
df_j

,upper,lower
0,30.509719,69.490281
1,12.951818,87.048182
2,8.076160,91.923840
3,10.133706,89.866294
4,2.851472,97.148528
5,24.837486,75.162514
6,22.129915,77.870085
7,20.712060,79.287940
8,15.716510,84.283490
9,28.888384,71.111616


In [ ]:
df_j.to_csv('pairTtest.csv')

In [ ]:
import researchpy

# Paired samples t-test
researchpy.ttest(df_j['upper'], df_j['lower'],
                            paired= True)

(  Variable     N       Mean         SD        SE  95% Conf.   Interval
 0    upper  37.0  25.708169  14.038671  2.307943  21.027443  30.388895
 1    lower  37.0  74.291831  14.038671  2.307943  69.611105  78.972557
 2     diff  37.0 -48.583663  28.077343  4.615887 -57.945115 -39.222210,
            Paired samples t-test  results
 0  Difference (upper - lower) =  -48.5837
 1          Degrees of freedom =   36.0000
 2                           t =  -10.5253
 3       Two side test p value =    0.0000
 4      Difference < 0 p value =    0.0000
 5      Difference > 0 p value =    1.0000
 6                   Cohen's d =   -3.4607
 7                   Hedge's g =   -3.4245
 8              Glass's delta1 =   -3.4607
 9            Point-Biserial r =   -0.8688)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(upper, lower, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-10.525315,36,two-sided,1.554368e-12,"[-57.95, -39.22]",3.460702,5.39e+09,1.0


In [ ]:
import math
math.sqrt(3.460702)

1.860296212972547

In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_upper: {Average(upper)}   mean_lower: {Average(lower)}')

mean_upper: 25.70816869964192   mean_lower: 74.29183130035807


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_upper: {np.std(upper)}   std_lower: {np.std(lower)}')

std_upper: 13.847660170553432   std_lower: 13.847660170553432


##Ipsilateral vs Contralateral hemi scenes

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

ipsi = []
contra = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = ((values[4] + values[14]) / values[0]) * 100 #hands in the ipsilateral scene
  ipsi.append(a)
  b = ((values[5] + values[13]) / values[0]) * 100 #hands in the contralateral scene
  contra.append(b)

In [ ]:
df_j = pd.DataFrame(list(zip(ipsi, contra)), columns = ['ipsi', 'contra'])

In [ ]:
p = []

for i in range(0, 37):
  p.append(i + 1)
p

In [ ]:
df_j['Participant'] = p

In [ ]:
df_j

,ipsi,contra,Participant
0,88.288594,11.711406,1
1,82.269216,17.730784,2
2,89.097042,10.902958,3
3,84.038971,15.961029,4
4,89.808316,10.191684,5
5,87.774692,12.225308,6
6,88.997463,11.002537,7
7,83.749621,16.250379,8
8,83.600759,16.399241,9
9,88.590761,11.409239,10


In [ ]:
df_j.to_csv('pairTtest1.csv')

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(ipsi, contra, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,65.81915,36,two-sided,4.078926e-39,"[74.13, 78.84]",21.641203,2.906e+35,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_ipsi: {Average(ipsi)}   mean_contra: {Average(contra)}')

mean_ipsi: 88.24324647859177   mean_contra: 11.756753521408214


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_ipsi: {np.std(ipsi)}   std_contra: {np.std(contra)}')

std_ipsi: 3.4862115261795754   std_contra: 3.4862115261795754


#Statistical differences between the hands for the various quadrants or hemifields.


##Contralateral hemi-scene - Left vs Right Hand

In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

overall_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/Hands_overall/*.csv'))

print("Total number of files: " + str(len(overall_counts)))

Total number of files: 37


In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_contra = []
R_contra = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[5] / values[1]) * 100 #left hand - contralateral hemi scene
  L_contra.append(a)
  b = (values[13] / values[10]) * 100 #right hand - contralateral hemi scene
  R_contra.append(b)

In [ ]:
df_j = pd.DataFrame(list(zip(L_contra, R_contra)), columns = ['L_contra', 'R_contra'])

In [ ]:
df_j

,L_contra,R_contra
0,19.206678,4.439968
1,9.510014,25.493853
2,14.788950,7.154158
3,15.696880,16.195205
4,7.488658,12.221532
5,17.962963,6.838891
6,18.461955,3.967426
7,27.652138,5.896754
8,30.653440,4.425574
9,18.379282,4.607283


In [ ]:
df_j.to_csv('pairTtest.csv')

In [ ]:
import researchpy

# Paired samples t-test
researchpy.ttest(df_j['L_contra'], df_j['R_contra'],
                            paired= True)

(   Variable     N       Mean         SD        SE  95% Conf.   Interval
 0  L_contra  37.0  16.173120   8.161855  1.341801  13.451822  18.894418
 1  R_contra  37.0   7.620733   5.312535  0.873375   5.849445   9.392020
 2      diff  37.0   8.552387  11.716004  1.926099   4.646077  12.458697,
                  Paired samples t-test  results
 0  Difference (L_contra - R_contra) =    8.5524
 1                Degrees of freedom =   36.0000
 2                                 t =    4.4403
 3             Two side test p value =    0.0001
 4            Difference < 0 p value =    0.0000
 5            Difference > 0 p value =    1.0000
 6                         Cohen's d =    1.2694
 7                         Hedge's g =    1.2562
 8                    Glass's delta1 =    1.0478
 9                  Point-Biserial r =    0.5949)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_contra, R_contra, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.440263,36,two-sided,0.000082,"[4.65, 12.46]",1.241964,297.115,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_L_contra: {Average(L_contra)}   mean_R_contra: {Average(R_contra)}')

mean_L_contra: 16.173119933384633   mean_R_contra: 7.620732604440766


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_contra: {np.std(L_contra)}   std_R_contra: {np.std(R_contra)}')

std_L_contra: 8.050803821971972   std_R_contra: 5.240252528667093


##Ipsilateral hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_ipsi = []
R_ipsi = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[4] / values[1]) * 100 #left hand - contralateral hemi scene
  L_ipsi.append(a)
  b = (values[14] / values[10]) * 100 #right hand - contralateral hemi scene
  R_ipsi.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_ipsi, R_ipsi, paired=True)

In [ ]:
#prints means under comparison
print(f'mean_L_ipsi: {Average(L_ipsi)}   mean_R_ipsi: {Average(R_ipsi)}')

In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_ipsi: {np.std(L_ipsi)}   std_R_ipsi: {np.std(R_ipsi)}')

##Lower hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_lower = []
R_lower = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[3] / values[1]) * 100 #left hand - contralateral hemi scene
  L_lower.append(a)
  b = (values[12] / values[10]) * 100 #right hand - contralateral hemi scene
  R_lower.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_lower, R_lower, paired=True)

In [ ]:
#prints means under comparison
print(f'mean_L_lower: {Average(L_lower)}   mean_R_lower: {Average(R_lower)}')

In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_lower: {np.std(L_lower)}   std_R_lower: {np.std(R_lower)}')

##Upper hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_upper = []
R_upper = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[2] / values[1]) * 100 #left hand - contralateral hemi scene
  L_upper.append(a)
  b = (values[11] / values[10]) * 100 #right hand - contralateral hemi scene
  R_upper.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_upper, R_upper, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-5.523811,36,two-sided,0.000003,"[-7.6, -3.52]",0.385978,6186.046,0.62738


In [ ]:
#prints means under comparison
print(f'mean_L_upper: {Average(L_upper)}   mean_R_upper: {Average(R_upper)}')

mean_L_upper: 22.85937119200265   mean_R_upper: 28.415930036616526


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_upper: {np.std(L_upper)}   std_R_upper: {np.std(R_upper)}')

std_L_upper: 12.903713207401552   std_R_upper: 15.387791700696932


##Contralateral upper quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_UR = []
R_UL = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[8] / values[1]) * 100 #left hand - Upper Right quadrant
  L_UR.append(a)
  b = (values[15] / values[10]) * 100 #right hand - Upper Left quadrant
  R_UL.append(b) 

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_UR, R_UL, paired=True)

In [ ]:
#prints means under comparison
print(f'mean_L_UR: {Average(L_UR)}   mean_R_UL: {Average(R_UL)}')

In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_UR: {np.std(L_UR)}   std_R_UL: {np.std(R_UL)}')

##Ipsilateral upper quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_UL = []
R_UR = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[6] / values[1]) * 100 #left hand - Upper Left quadrant
  L_UL.append(a)
  b = (values[17] / values[10]) * 100 #right hand - Upper Right quadrant
  R_UR.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_UL, R_UR, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-6.61228,36,two-sided,1.064765e-07,"[-9.14, -4.85]",0.514581,1.396e+05,0.861173


In [ ]:
#prints means under comparison
print(f'mean_L_UL: {Average(L_UL)}   mean_R_UR: {Average(R_UR)}')

mean_L_UL: 19.597771199802388   mean_R_UR: 26.594039481839918


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_UL: {np.std(L_UL)}   std_R_UR: {np.std(R_UR)}')

std_L_UL: 12.011649781774581   std_R_UR: 14.677649119517707


##Contralateral lower quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_LR = []
R_LL = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[9] / values[1]) * 100 #left hand - Lower Right quadrant
  L_LR.append(a)
  b = (values[16] / values[10]) * 100 #right hand - Lower Left quadrant
  R_LL.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_LR, R_LL, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.400129,36,two-sided,0.000092,"[3.83, 10.39]",1.166151,266.361,1.0


In [ ]:
#prints means under comparison
print(f'mean_L_LR: {Average(L_LR)}   mean_R_LL: {Average(R_LL)}')

mean_L_LR: 12.911519941184375   mean_R_LL: 5.79884204966415


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_LR: {np.std(L_LR)}   std_R_LL: {np.std(R_LL)}')

std_L_LR: 7.12690553309151   std_R_LL: 4.647443514374768


##Ipsilateral lower quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_LL = []
R_LR = []

for i in overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[7] / values[1]) * 100 #left hand - Lower Left quadrant
  L_LL.append(a)
  b = (values[18] / values[10]) * 100 #right hand - Lower Right quadrant
  R_LR.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_LL, R_LR, paired=True)

In [ ]:
#prints means under comparison
print(f'mean_L_LL: {Average(L_LL)}   mean_R_LR: {Average(R_LR)}')

In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_LL: {np.std(L_LL)}   std_R_LR: {np.std(R_LR)}')

#Statistical differences between the different hand interactions

In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

portable_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/Hands_Portable/*.csv'))

print("Total number of files: " + str(len(portable_counts)))

Total number of files: 37


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

stationary_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/Hands_Stationary/*.csv'))

print("Total number of files: " + str(len(stationary_counts)))

Total number of files: 37


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

noContact_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/Hands_NoContact/*.csv'))

print("Total number of files: " + str(len(noContact_counts)))

Total number of files: 37


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

self_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/Hands_Self/*.csv'))

print("Total number of files: " + str(len(self_counts)))

Total number of files: 37


##Stationary objects vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_stat = []
R_low_noC = []

for stat, noC in zip(stationary_counts, noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - stationary ojects
  R_low_stat.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_stat, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-8.805643,36,two-sided,1.656661e-10,"[-19.06, -11.92]",1.200852,6.278e+07,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))

In [ ]:
#prints means under comparison
print(f'mean_R_lower_stationary: {Average(R_low_stat)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_stationary: 64.17856647376813   mean_R_lower_noContact: 79.6709253681016


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_R_lower_stationary: {np.std(R_low_stat)}   std_R_lower_noContact: {np.std(R_low_noC)}')

std_R_lower_stationary: 15.415555784680958   std_R_lower_noContact: 9.286692373073098


##Portable objects vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_port = []
R_low_noC = []

for port, noC in zip(portable_counts, noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[13] / values_port[11]) * 100 #right hand - lower - portable ojects
  R_low_port.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_port, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-5.180928,36,two-sided,0.000009,"[-12.69, -5.55]",0.666585,2335.057,0.976392


In [ ]:
#prints means under comparison
print(f'mean_R_lower_portable: {Average(R_low_port)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_portable: 70.55337189323097   mean_R_lower_noContact: 79.6709253681016


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_R_lower_portable: {np.std(R_low_port)}   std_R_lower_noContact: {np.std(R_low_noC)}')

std_R_lower_portable: 16.66792888697564   std_R_lower_noContact: 9.286692373073098


##Self-contact vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_noC = []

for selfie, noC in zip(self_counts, noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - self
  R_low_self.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.684916,36,two-sided,0.497783,"[-2.6, 5.26]",0.101638,0.22,0.092426


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_self: 80.99878461718818   mean_R_lower_noContact: 79.6709253681016


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_R_lower_self: {np.std(R_low_self)}   std_R_lower_noContact: {np.std(R_low_noC)}')

std_R_lower_self: 15.681173449543648   std_R_lower_noContact: 9.286692373073098


##Self-contact vs Portable objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_port = []

for selfie, port in zip(self_counts, portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - stationary ojects
  R_low_self.append(a)
  b = (values_port[13] / values_port[11]) * 100 #right hand - lower - port
  R_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.01217,36,two-sided,0.000014,"[6.22, 14.67]",0.63671,1450.806,0.964677


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_portable: {Average(R_low_port)}')

mean_R_lower_self: 80.99878461718818   mean_R_lower_portable: 70.55337189323097


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_R_lower_self: {np.std(R_low_self)}   std_R_lower_portable: {np.std(R_low_port)}')

std_R_lower_self: 15.681173449543648   std_R_lower_portable: 16.66792888697564


##Self-contact vs Stationary objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_stat = []

for selfie, stat in zip(self_counts, stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - stationary ojects
  R_low_self.append(a)
  b = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - port
  R_low_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,7.108096,36,two-sided,2.369591e-08,"[12.02, 21.62]",1.067042,5.728e+05,1.0


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_stationary: {Average(R_low_stat)}')

mean_R_lower_self: 80.99878461718818   mean_R_lower_stationary: 64.17856647376813


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_R_lower_self: {np.std(R_low_self)}   std_R_lower_stationary: {np.std(R_low_stat)}')

std_R_lower_self: 15.681173449543648   std_R_lower_stationary: 15.415555784680958


##Stationary vs Portable objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_stat = []
R_low_port = []

for stat, port in zip(stationary_counts, portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - stationary ojects
  R_low_stat.append(a)
  b = (values_port[13] / values_port[11]) * 100 #right hand - lower - port
  R_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_stat, R_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-3.101274,36,two-sided,0.003734,"[-10.54, -2.21]",0.391683,9.799,0.64008


In [ ]:
#prints means under comparison
print(f'mean_R_lower_stationary: {Average(R_low_stat)}   mean_R_lower_portable: {Average(R_low_port)}')

mean_R_lower_stationary: 64.17856647376813   mean_R_lower_portable: 70.55337189323097


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_R_lower_stationary: {np.std(R_low_stat)}   std_R_lower_portable: {np.std(R_low_port)}')

std_R_lower_stationary: 15.415555784680958   std_R_lower_portable: 16.66792888697564


##Stationary objects vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_stat = []
R_up_noC = []

for stat, noC in zip(stationary_counts, noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - stationary ojects
  R_up_stat.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_stat, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,8.805643,36,two-sided,1.656661e-10,"[11.92, 19.06]",1.200852,6.278e+07,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_R_upper_stationary: {Average(R_up_stat)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_stationary: 35.821433526231885   mean_R_upper_noContact: 20.329074631898397


##Portable objects vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_port = []
R_up_noC = []

for port, noC in zip(portable_counts, noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[12] / values_port[11]) * 100 #right hand - upper - portable ojects
  R_up_port.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_port, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.180928,36,two-sided,0.000009,"[5.55, 12.69]",0.666585,2335.057,0.976392


In [ ]:
#prints means under comparison
print(f'mean_R_upper_portable: {Average(R_up_port)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_portable: 29.446628106769015   mean_R_upper_noContact: 20.329074631898397


##Self-contact vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_noC = []

for selfie, noC in zip(self_counts, noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - self
  R_up_self.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.684916,36,two-sided,0.497783,"[-5.26, 2.6]",0.101638,0.22,0.092426


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_self: 19.00121538281182   mean_R_upper_noContact: 20.329074631898397


##Self-contact vs Portable objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_port = []

for selfie, port in zip(self_counts, portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - stationary ojects
  R_up_self.append(a)
  b = (values_port[12] / values_port[11]) * 100 #right hand - upper - port
  R_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-5.01217,36,two-sided,0.000014,"[-14.67, -6.22]",0.63671,1450.806,0.964677


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_portable: {Average(R_up_port)}')

mean_R_upper_self: 19.00121538281182   mean_R_upper_portable: 29.446628106769015


##Self-contact vs Stationary objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_stat = []

for selfie, stat in zip(self_counts, stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - stationary ojects
  R_up_self.append(a)
  b = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - port
  R_up_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-7.108096,36,two-sided,2.369591e-08,"[-21.62, -12.02]",1.067042,5.728e+05,1.0


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_stationary: {Average(R_up_stat)}')

mean_R_upper_self: 19.00121538281182   mean_R_upper_stationary: 35.821433526231885


##Stationary vs Portable objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_stat = []
R_up_port = []

for stat, port in zip(stationary_counts, portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - stationary ojects
  R_up_stat.append(a)
  b = (values_port[12] / values_port[11]) * 100 #right hand - upper - port
  R_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_stat, R_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.101274,36,two-sided,0.003734,"[2.21, 10.54]",0.391683,9.799,0.64008


In [ ]:
#prints means under comparison
print(f'mean_R_upper_stationary: {Average(R_up_stat)}   mean_R_upper_portable: {Average(R_up_port)}')

mean_R_upper_stationary: 35.821433526231885   mean_R_upper_portable: 29.446628106769015


##Stationary objects vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_stat = []
L_low_noC = []

for stat, noC in zip(stationary_counts, noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - stationary ojects
  L_low_stat.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_stat, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-9.850688,36,two-sided,9.271306e-12,"[-14.75, -9.71]",1.292627,9.786e+08,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))

In [ ]:
#prints means under comparison
print(f'mean_L_lower_stationary: {Average(L_low_stat)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_stationary: 74.29541295288641   mean_L_lower_noContact: 86.52845114605958


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_lower_stationary: {np.std(L_low_stat)}   std_L_lower_noContact: {np.std(L_low_noC)}')

std_L_lower_stationary: 11.719233948527057   std_L_lower_noContact: 6.0779668032851815


##Portable objects vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_port = []
L_low_noC = []

for port, noC in zip(portable_counts, noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[4] / values_port[2]) * 100 #left hand - lower - portable ojects
  L_low_port.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_port, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-6.049445,36,two-sided,5.969341e-07,"[-15.16, -7.55]",0.976801,2.784e+04,0.999933


In [ ]:
#prints means under comparison
print(f'mean_L_lower_portable: {Average(L_low_port)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_portable: 75.17311093444812   mean_L_lower_noContact: 86.52845114605958


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_lower_portable: {np.std(L_low_port)}   std_L_lower_noContact: {np.std(L_low_noC)}')

std_L_lower_portable: 15.034501236367724   std_L_lower_noContact: 6.0779668032851815


##Self-contact vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_noC = []

for selfie, noC in zip(self_counts, noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self
  L_low_self.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.05465,36,two-sided,0.047226,"[-8.2, -0.05]",0.352713,1.155,0.550981


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_self: 82.4002203650508   mean_L_lower_noContact: 86.52845114605958


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_lower_self: {np.std(L_low_self)}   std_L_lower_noContact: {np.std(L_low_noC)}')

std_L_lower_self: 15.153593999456303   std_L_lower_noContact: 6.0779668032851815


##Self-contact vs Portable objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_port = []

for selfie, port in zip(self_counts, portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self ojects
  L_low_self.append(a)
  b = (values_port[4] / values_port[2]) * 100 #left hand - lower - port
  L_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.704894,36,two-sided,0.000706,"[3.27, 11.18]",0.472287,42.58,0.798271


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_portable: {Average(L_low_port)}')

mean_L_lower_self: 82.4002203650508   mean_L_lower_portable: 75.17311093444812


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_lower_self: {np.std(L_low_self)}   std_L_lower_portable: {np.std(L_low_port)}')

std_L_lower_self: 15.153593999456303   std_L_lower_portable: 15.034501236367724


##Self-contact vs Stationary objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_stat = []

for selfie, stat in zip(self_counts, stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self
  L_low_self.append(a)
  b = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - port
  L_low_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.47577,36,two-sided,0.001347,"[3.38, 12.83]",0.59019,23.993,0.937326


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_stationary: {Average(L_low_stat)}')

mean_L_lower_self: 82.4002203650508   mean_L_lower_stationary: 74.29541295288641


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_lower_self: {np.std(L_low_self)}   std_L_lower_stationary: {np.std(L_low_stat)}')

std_L_lower_self: 15.153593999456303   std_L_lower_stationary: 11.719233948527057


##Stationary vs Portable objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_stat = []
L_low_port = []

for stat, port in zip(stationary_counts, portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - stationary ojects
  L_low_stat.append(a)
  b = (values_port[4] / values_port[2]) * 100 #left hand - lower - port
  L_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_stat, L_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.437004,36,two-sided,0.664718,"[-4.95, 3.2]",0.064229,0.193,0.066734


In [ ]:
#prints means under comparison
print(f'mean_L_lower_stationary: {Average(L_low_stat)}   mean_L_lower_portable: {Average(L_low_port)}')

mean_L_lower_stationary: 74.29541295288641   mean_L_lower_portable: 75.17311093444812


In [ ]:
import numpy as np

#prints std under comparison
print(f'std_L_lower_stationary: {np.std(L_low_stat)}   std_L_lower_portable: {np.std(L_low_port)}')

std_L_lower_stationary: 11.719233948527057   std_L_lower_portable: 15.034501236367724


##Stationary objects vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_stat = []
L_up_noC = []

for stat, noC in zip(stationary_counts, noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - stationary ojects
  L_up_stat.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_stat, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,9.850688,36,two-sided,9.271306e-12,"[9.71, 14.75]",1.292627,9.786e+08,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))

In [ ]:
#prints means under comparison
print(f'mean_L_upper_stationary: {Average(L_up_stat)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_stationary: 25.7045870471136   mean_L_upper_noContact: 13.471548853940433


##Portable objects vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_port = []
L_up_noC = []

for port, noC in zip(portable_counts, noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[3] / values_port[2]) * 100 #left hand - upper - portable ojects
  L_up_port.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_port, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,6.049445,36,two-sided,5.969341e-07,"[7.55, 15.16]",0.976801,2.784e+04,0.999933


In [ ]:
#prints means under comparison
print(f'mean_L_upper_portable: {Average(L_up_port)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_portable: 24.826889065551892   mean_L_upper_noContact: 13.471548853940433


##Self-contact vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_noC = []

for selfie, noC in zip(self_counts, noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self
  L_up_self.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.05465,36,two-sided,0.047226,"[0.05, 8.2]",0.352713,1.155,0.550981


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_self: 17.59977963494921   mean_L_upper_noContact: 13.471548853940433


##Self-contact vs Portable objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_port = []

for selfie, port in zip(self_counts, portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self ojects
  L_up_self.append(a)
  b = (values_port[3] / values_port[2]) * 100 #left hand - upper - port
  L_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-3.704894,36,two-sided,0.000706,"[-11.18, -3.27]",0.472287,42.58,0.798271


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_portable: {Average(L_up_port)}')

mean_L_upper_self: 17.59977963494921   mean_L_upper_portable: 24.826889065551892


##Self-contact vs Stationary objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_stat = []

for selfie, stat in zip(self_counts, stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self
  L_up_self.append(a)
  b = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - port
  L_up_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-3.47577,36,two-sided,0.001347,"[-12.83, -3.38]",0.59019,23.993,0.937326


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_stationary: {Average(L_up_stat)}')

mean_L_upper_self: 17.59977963494921   mean_L_upper_stationary: 25.7045870471136


##Stationary vs Portable objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_stat = []
L_up_port = []

for stat, port in zip(stationary_counts, portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - stationary ojects
  L_up_stat.append(a)
  b = (values_port[3] / values_port[2]) * 100 #left hand - upper - port
  L_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_stat, L_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.437004,36,two-sided,0.664718,"[-3.2, 4.95]",0.064229,0.193,0.066734


In [ ]:
#prints means under comparison
print(f'mean_L_upper_stationary: {Average(L_up_stat)}   mean_L_upper_portable: {Average(L_up_port)}')

mean_L_upper_stationary: 25.7045870471136   mean_L_upper_portable: 24.826889065551892


#Left handers

#Statistical differences between the hands for the various quadrants or hemifields.


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

L_overall_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/LeftHanders/Hands_overall/*.csv'))

print("Total number of files: " + str(len(L_overall_counts)))

Total number of files: 5


##Contralateral hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_contra = []
R_contra = []

for i in L_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[5] / values[1]) * 100 #left hand - contralateral hemi scene
  L_contra.append(a)
  b = (values[13] / values[10]) * 100 #right hand - contralateral hemi scene
  R_contra.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_contra, R_contra, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.924469,4,two-sided,0.407564,"[-8.1, 16.19]",0.60061,0.549,0.18095


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_L_contra: {Average(L_contra)}   mean_R_contra: {Average(R_contra)}')

mean_L_contra: 13.39367355965643   mean_R_contra: 9.349600662591778


##Ipsilateral hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_ipsi = []
R_ipsi = []

for i in L_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[4] / values[1]) * 100 #left hand - contralateral hemi scene
  L_ipsi.append(a)
  b = (values[14] / values[10]) * 100 #right hand - contralateral hemi scene
  R_ipsi.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_ipsi, R_ipsi, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.924469,4,two-sided,0.407564,"[-16.19, 8.1]",0.60061,0.549,0.18095


In [ ]:
#prints means under comparison
print(f'mean_L_ipsi: {Average(L_ipsi)}   mean_R_ipsi: {Average(R_ipsi)}')

mean_L_ipsi: 86.60632644034357   mean_R_ipsi: 90.65039933740823


##Lower hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_lower = []
R_lower = []

for i in L_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[3] / values[1]) * 100 #left hand - contralateral hemi scene
  L_lower.append(a)
  b = (values[12] / values[10]) * 100 #right hand - contralateral hemi scene
  R_lower.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_lower, R_lower, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.856435,4,two-sided,0.136965,"[-2.13, 10.74]",0.387022,1.096,0.103946


In [ ]:
#prints means under comparison
print(f'mean_L_lower: {Average(L_lower)}   mean_R_lower: {Average(R_lower)}')

mean_L_lower: 78.71842650228066   mean_R_lower: 74.4140114222259


##Upper hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_upper = []
R_upper = []

for i in L_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[2] / values[1]) * 100 #left hand - contralateral hemi scene
  L_upper.append(a)
  b = (values[11] / values[10]) * 100 #right hand - contralateral hemi scene
  R_upper.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_upper, R_upper, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.856435,4,two-sided,0.136965,"[-10.74, 2.13]",0.387022,1.096,0.103946


In [ ]:
#prints means under comparison
print(f'mean_L_upper: {Average(L_lower)}   mean_R_upper: {Average(R_lower)}')

mean_L_upper: 78.71842650228066   mean_R_upper: 74.4140114222259


##Contralateral upper quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_UR = []
R_UL = []

for i in L_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[8] / values[1]) * 100 #left hand - Upper Right quadrant
  L_UR.append(a)
  b = (values[15] / values[10]) * 100 #right hand - Upper Left quadrant
  R_UL.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_UR, R_UL, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.556134,4,two-sided,0.19466,"[-3.32, 0.93]",1.048336,0.866,0.432258


In [ ]:
#prints means under comparison
print(f'mean_L_UR: {Average(L_UR)}   mean_R_UL: {Average(R_UL)}')

mean_L_UR: 1.642988252176988   mean_R_UL: 2.834126462774633


##Ipsilateral upper quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_UL = []
R_UR = []

for i in L_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[6] / values[1]) * 100 #left hand - Upper Left quadrant
  L_UL.append(a)
  b = (values[17] / values[10]) * 100 #right hand - Upper Right quadrant
  R_UR.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_UL, R_UR, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.747901,4,two-sided,0.155397,"[-8.06, 1.83]",0.294752,1.006,0.081135


In [ ]:
#prints means under comparison
print(f'mean_L_UL: {Average(L_UL)}   mean_R_UR: {Average(R_UR)}')

mean_L_UL: 19.63858524554235   mean_R_UR: 22.75186211499946


##Contralateral lower quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_LR = []
R_LL = []

for i in L_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[9] / values[1]) * 100 #left hand - Lower Right quadrant
  L_LR.append(a)
  b = (values[16] / values[10]) * 100 #right hand - Lower Left quadrant
  R_LL.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_LR, R_LL, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.275853,4,two-sided,0.27106,"[-6.16, 16.63]",0.839089,0.699,0.303258


In [ ]:
#prints means under comparison
print(f'mean_L_LR: {Average(L_LR)}   mean_R_LL: {Average(R_LL)}')

mean_L_LR: 11.750685307479442   mean_R_LL: 6.515474199817144


##Ipsilateral lower quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_LL = []
R_LR = []

for i in L_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[7] / values[1]) * 100 #left hand - Lower Left quadrant
  L_LL.append(a)
  b = (values[18] / values[10]) * 100 #right hand - Lower Right quadrant
  R_LR.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_LL, R_LR, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.235807,4,two-sided,0.825164,"[-11.89, 10.03]",0.082816,0.407,0.052438


In [ ]:
#prints means under comparison
print(f'mean_L_LL: {Average(L_LL)}   mean_R_LR: {Average(R_LR)}')

mean_L_LL: 66.96774119480122   mean_R_LR: 67.89853722240875


#Statistical differences between the different hand interactions

In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

L_portable_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/LeftHanders/Hands_Portable/*.csv'))

print("Total number of files: " + str(len(L_portable_counts)))

Total number of files: 5


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

L_stationary_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/LeftHanders/Hands_Stationary/*.csv'))

print("Total number of files: " + str(len(L_stationary_counts)))

Total number of files: 5


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

L_noContact_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/LeftHanders/Hands_NoContact/*.csv'))

print("Total number of files: " + str(len(L_noContact_counts)))

Total number of files: 5


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

L_self_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/LeftHanders/Hands_Self/*.csv'))

print("Total number of files: " + str(len(L_self_counts)))

Total number of files: 5


##Stationary objects vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_stat = []
R_low_noC = []

for stat, noC in zip(L_stationary_counts, L_noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - stationary ojects
  R_low_stat.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_stat, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.459227,4,two-sided,0.218269,"[-15.48, 4.81]",0.467735,0.803,0.129106


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_R_lower_stationary: {Average(R_low_stat)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_stationary: 78.16528843023887   mean_R_lower_noContact: 83.49801253923853


##Portable objects vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_port = []
R_low_noC = []

for port, noC in zip(L_portable_counts, L_noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[13] / values_port[11]) * 100 #right hand - lower - portable ojects
  R_low_port.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_port, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.897271,4,two-sided,0.044239,"[-22.19, -0.47]",1.003454,2.421,0.403581


In [ ]:
#prints means under comparison
print(f'mean_R_lower_portable: {Average(R_low_port)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_portable: 72.16576729594297   mean_R_lower_noContact: 83.49801253923853


##Self-contact vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_noC = []

for selfie, noC in zip(L_self_counts, L_noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - self
  R_low_self.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.450383,4,two-sided,0.675767,"[-18.22, 13.13]",0.260691,0.432,0.074313


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_self: 80.95498575628582   mean_R_lower_noContact: 83.49801253923853


##Self-contact vs Portable objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_port = []

for selfie, port in zip(L_self_counts, L_portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - stationary ojects
  R_low_self.append(a)
  b = (values_port[13] / values_port[11]) * 100 #right hand - lower - port
  R_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.510977,4,two-sided,0.20532,"[-7.36, 24.94]",0.664539,0.836,0.21031


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_portable: {Average(R_low_port)}')

mean_R_lower_self: 80.95498575628582   mean_R_lower_portable: 72.16576729594297


##Self-contact vs Stationary objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_stat = []

for selfie, stat in zip(L_self_counts, L_stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - stationary ojects
  R_low_self.append(a)
  b = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - port
  R_low_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.345729,4,two-sided,0.746964,"[-19.61, 25.19]",0.209463,0.418,0.065658


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_stationary: {Average(R_low_stat)}')

mean_R_lower_self: 80.95498575628582   mean_R_lower_stationary: 78.16528843023887


##Stationary vs Portable objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_stat = []
R_low_port = []

for stat, port in zip(L_stationary_counts, L_portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - stationary ojects
  R_low_stat.append(a)
  b = (values_port[13] / values_port[11]) * 100 #right hand - lower - port
  R_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_stat, R_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.360629,4,two-sided,0.245252,"[-6.24, 18.24]",0.414247,0.745,0.11189


In [ ]:
#prints means under comparison
print(f'mean_R_lower_stationary: {Average(R_low_stat)}   mean_R_lower_portable: {Average(R_low_port)}')

mean_R_lower_stationary: 78.16528843023887   mean_R_lower_portable: 72.16576729594297


##Stationary objects vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_stat = []
R_up_noC = []

for stat, noC in zip(L_stationary_counts, L_noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - stationary ojects
  R_up_stat.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_stat, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.459227,4,two-sided,0.218269,"[-4.81, 15.48]",0.467735,0.803,0.129106


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_R_upper_stationary: {Average(R_up_stat)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_stationary: 21.834711569761115   mean_R_upper_noContact: 16.501987460761477


##Portable objects vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_port = []
R_up_noC = []

for port, noC in zip(L_portable_counts, L_noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[12] / values_port[11]) * 100 #right hand - upper - portable ojects
  R_up_port.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_port, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.897271,4,two-sided,0.044239,"[0.47, 22.19]",1.003454,2.421,0.403581


In [ ]:
#prints means under comparison
print(f'mean_R_upper_portable: {Average(R_up_port)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_portable: 27.83423270405703   mean_R_upper_noContact: 16.501987460761477


##Self-contact vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_noC = []

for selfie, noC in zip(L_self_counts, L_noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - self
  R_up_self.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.450383,4,two-sided,0.675767,"[-13.13, 18.22]",0.260691,0.432,0.074313


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_self: 19.045014243714167   mean_R_upper_noContact: 16.501987460761477


##Self-contact vs Portable objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_port = []

for selfie, port in zip(L_self_counts, L_portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - stationary ojects
  R_up_self.append(a)
  b = (values_port[12] / values_port[11]) * 100 #right hand - upper - port
  R_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.510977,4,two-sided,0.20532,"[-24.94, 7.36]",0.664539,0.836,0.21031


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_portable: {Average(R_up_port)}')

mean_R_upper_self: 19.045014243714167   mean_R_upper_portable: 27.83423270405703


##Self-contact vs Stationary objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_stat = []

for selfie, stat in zip(L_self_counts, L_stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - stationary ojects
  R_up_self.append(a)
  b = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - port
  R_up_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.345729,4,two-sided,0.746964,"[-25.19, 19.61]",0.209463,0.418,0.065658


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_stationary: {Average(R_up_stat)}')

mean_R_upper_self: 19.045014243714167   mean_R_upper_stationary: 21.834711569761115


##Stationary vs Portable objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_stat = []
R_up_port = []

for stat, port in zip(L_stationary_counts, L_portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - stationary ojects
  R_up_stat.append(a)
  b = (values_port[12] / values_port[11]) * 100 #right hand - upper - port
  R_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_stat, R_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.360629,4,two-sided,0.245252,"[-18.24, 6.24]",0.414247,0.745,0.11189


In [ ]:
#prints means under comparison
print(f'mean_R_upper_stationary: {Average(R_up_stat)}   mean_R_upper_portable: {Average(R_up_port)}')

mean_R_upper_stationary: 21.834711569761115   mean_R_upper_portable: 27.83423270405703


##Stationary objects vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_stat = []
L_low_noC = []

for stat, noC in zip(L_stationary_counts, L_noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - stationary ojects
  L_low_stat.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_stat, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.36352,4,two-sided,0.077365,"[-17.69, 1.42]",1.824198,1.629,0.856091


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_L_lower_stationary: {Average(L_low_stat)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_stationary: 79.26434133606502   mean_L_lower_noContact: 87.40030265577835


##Portable objects vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_port = []
L_low_noC = []

for port, noC in zip(L_portable_counts, L_noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[4] / values_port[2]) * 100 #left hand - lower - portable ojects
  L_low_port.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_port, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-2.271067,4,two-sided,0.085627,"[-22.49, 2.25]",1.282687,1.517,0.582794


In [ ]:
#prints means under comparison
print(f'mean_L_lower_portable: {Average(L_low_port)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_portable: 77.28221852588196   mean_L_lower_noContact: 87.40030265577835


##Self-contact vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_noC = []

for selfie, noC in zip(L_self_counts, L_noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self
  L_low_self.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.036048,4,two-sided,0.358702,"[-22.37, 10.21]",0.630651,0.59,0.194404


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_self: 81.32072351409688   mean_L_lower_noContact: 87.40030265577835


##Self-contact vs Portable objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_port = []

for selfie, port in zip(L_self_counts, L_portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self ojects
  L_low_self.append(a)
  b = (values_port[4] / values_port[2]) * 100 #left hand - lower - port
  L_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.671729,4,two-sided,0.538558,"[-12.65, 20.73]",0.332078,0.475,0.089599


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_portable: {Average(L_low_port)}')

mean_L_lower_self: 81.32072351409688   mean_L_lower_portable: 77.28221852588196


##Self-contact vs Stationary objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_stat = []

for selfie, stat in zip(L_self_counts, L_stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self
  L_low_self.append(a)
  b = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - port
  L_low_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.358802,4,two-sided,0.73788,"[-13.86, 17.97]",0.200143,0.419,0.06429


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_stationary: {Average(L_low_stat)}')

mean_L_lower_self: 81.32072351409688   mean_L_lower_stationary: 79.26434133606502


##Stationary vs Portable objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_stat = []
L_low_port = []

for stat, port in zip(L_stationary_counts, L_portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - stationary ojects
  L_low_stat.append(a)
  b = (values_port[4] / values_port[2]) * 100 #left hand - lower - port
  L_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_stat, L_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.341184,4,two-sided,0.750134,"[-14.15, 18.11]",0.229094,0.417,0.068747


In [ ]:
#prints means under comparison
print(f'mean_L_lower_stationary: {Average(L_low_stat)}   mean_L_lower_portable: {Average(L_low_port)}')

mean_L_lower_stationary: 79.26434133606502   mean_L_lower_portable: 77.28221852588196


##Stationary objects vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_stat = []
L_up_noC = []

for stat, noC in zip(L_stationary_counts, L_noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - stationary ojects
  L_up_stat.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_stat, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.36352,4,two-sided,0.077365,"[-1.42, 17.69]",1.824198,1.629,0.856091


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))

In [ ]:
#prints means under comparison
print(f'mean_L_upper_stationary: {Average(L_up_stat)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_stationary: 20.735658663934984   mean_L_upper_noContact: 12.59969734422165


##Portable objects vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_port = []
L_up_noC = []

for port, noC in zip(L_portable_counts, L_noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[3] / values_port[2]) * 100 #left hand - upper - portable ojects
  L_up_port.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_port, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.271067,4,two-sided,0.085627,"[-2.25, 22.49]",1.282687,1.517,0.582794


In [ ]:
#prints means under comparison
print(f'mean_L_upper_portable: {Average(L_up_port)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_portable: 22.71778147411805   mean_L_upper_noContact: 12.59969734422165


##Self-contact vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_noC = []

for selfie, noC in zip(L_self_counts, L_noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self
  L_up_self.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.036048,4,two-sided,0.358702,"[-10.21, 22.37]",0.630651,0.59,0.194404


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_self: 18.679276485903117   mean_L_upper_noContact: 12.59969734422165


##Self-contact vs Portable objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_port = []

for selfie, port in zip(L_self_counts, L_portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self ojects
  L_up_self.append(a)
  b = (values_port[3] / values_port[2]) * 100 #left hand - upper - port
  L_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.671729,4,two-sided,0.538558,"[-20.73, 12.65]",0.332078,0.475,0.089599


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_portable: {Average(L_up_port)}')

mean_L_upper_self: 18.679276485903117   mean_L_upper_portable: 22.71778147411805


##Self-contact vs Stationary objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_stat = []

for selfie, stat in zip(L_self_counts, L_stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self
  L_up_self.append(a)
  b = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - port
  L_up_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.358802,4,two-sided,0.73788,"[-17.97, 13.86]",0.200143,0.419,0.06429


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_stationary: {Average(L_up_stat)}')

mean_L_upper_self: 18.679276485903117   mean_L_upper_stationary: 20.735658663934984


##Stationary vs Portable objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_stat = []
L_up_port = []

for stat, port in zip(L_stationary_counts, L_portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - stationary ojects
  L_up_stat.append(a)
  b = (values_port[3] / values_port[2]) * 100 #left hand - upper - port
  L_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_stat, L_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.341184,4,two-sided,0.750134,"[-18.11, 14.15]",0.229094,0.417,0.068747


In [ ]:
#prints means under comparison
print(f'mean_L_upper_stationary: {Average(L_up_stat)}   mean_L_upper_portable: {Average(L_up_port)}')

mean_L_upper_stationary: 20.735658663934984   mean_L_upper_portable: 22.71778147411805


#Right Handers

#Statistical differences between the hands for the various quadrants or hemifields.


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

R_overall_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/RightHanders/Hands_overall/*.csv'))

print("Total number of files: " + str(len(R_overall_counts)))

Total number of files: 32


##Contralateral hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_contra = []
R_contra = []

for i in R_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[5] / values[1]) * 100 #left hand - contralateral hemi scene
  L_contra.append(a)
  b = (values[13] / values[10]) * 100 #right hand - contralateral hemi scene
  R_contra.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_contra, R_contra, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.374838,31,two-sided,0.000128,"[4.94, 13.57]",1.333072,205.352,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_L_contra: {Average(L_contra)}   mean_R_contra: {Average(R_contra)}')

mean_L_contra: 16.607408429279662   mean_R_contra: 7.350596970354668


##Ipsilateral hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_ipsi = []
R_ipsi = []

for i in R_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[4] / values[1]) * 100 #left hand - contralateral hemi scene
  L_ipsi.append(a)
  b = (values[14] / values[10]) * 100 #right hand - contralateral hemi scene
  R_ipsi.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_ipsi, R_ipsi, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.374838,31,two-sided,0.000128,"[-13.57, -4.94]",1.333072,205.352,1.0


In [ ]:
#prints means under comparison
print(f'mean_L_ipsi: {Average(L_ipsi)}   mean_R_ipsi: {Average(R_ipsi)}')

mean_L_ipsi: 83.39259157072031   mean_R_ipsi: 92.64940302964531


##Lower hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_lower = []
R_lower = []

for i in R_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[3] / values[1]) * 100 #left hand - contralateral hemi scene
  L_lower.append(a)
  b = (values[12] / values[10]) * 100 #right hand - contralateral hemi scene
  R_lower.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_lower, R_lower, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.163171,31,two-sided,0.000013,"[3.48, 8.02]",0.384564,1592.803,0.55877


In [ ]:
#prints means under comparison
print(f'mean_L_lower: {Average(L_lower)}   mean_R_lower: {Average(R_lower)}')

mean_L_lower: 76.8940979182656   mean_R_lower: 71.14189161043934


##Upper hemi-scene - Left vs Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_upper = []
R_upper = []

for i in R_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[2] / values[1]) * 100 #left hand - contralateral hemi scene
  L_upper.append(a)
  b = (values[11] / values[10]) * 100 #right hand - contralateral hemi scene
  R_upper.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_upper, R_upper, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-5.163171,31,two-sided,0.000013,"[-8.02, -3.48]",0.384564,1592.803,0.55877


In [ ]:
#prints means under comparison
print(f'mean_L_upper: {Average(L_lower)}   mean_R_upper: {Average(R_lower)}')

mean_L_upper: 76.8940979182656   mean_R_upper: 71.14189161043934


##Contralateral upper quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_UR = []
R_UL = []

for i in R_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[8] / values[1]) * 100 #left hand - Upper Right quadrant
  L_UR.append(a)
  b = (values[15] / values[10]) * 100 #right hand - Upper Left quadrant
  R_UL.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_UR, R_UL, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.955729,31,two-sided,0.000413,"[0.9, 2.81]",1.051357,71.469,0.999926


In [ ]:
#prints means under comparison
print(f'mean_L_UR: {Average(L_UR)}   mean_R_UL: {Average(R_UL)}')

mean_L_UR: 3.5145080765788954   mean_R_UL: 1.6637286941519238


##Ipsilateral upper quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_UL = []
R_UR = []

for i in R_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[6] / values[1]) * 100 #left hand - Upper Left quadrant
  L_UL.append(a)
  b = (values[17] / values[10]) * 100 #right hand - Upper Right quadrant
  R_UR.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_UL, R_UR, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-6.540244,31,two-sided,2.664153e-07,"[-9.97, -5.23]",0.539103,5.997e+04,0.839879


In [ ]:
#prints means under comparison
print(f'mean_L_UL: {Average(L_UL)}   mean_R_UR: {Average(R_UR)}')

mean_L_UL: 19.591394005155518   mean_R_UR: 27.194379695408745


##Contralateral lower quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_LR = []
R_LL = []

for i in R_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[9] / values[1]) * 100 #left hand - Lower Right quadrant
  L_LR.append(a)
  b = (values[16] / values[10]) * 100 #right hand - Lower Left quadrant
  R_LL.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_LR, R_LL, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.173785,31,two-sided,0.000225,"[3.79, 11.02]",1.201338,123.276,1.0


In [ ]:
#prints means under comparison
print(f'mean_L_LR: {Average(L_LR)}   mean_R_LL: {Average(R_LL)}')

mean_L_LR: 13.09290035270077   mean_R_LL: 5.686868276202745


##Ipsilateral lower quadrant - Left vs Right hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_LL = []
R_LR = []

for i in R_overall_counts:
  df = pd.read_csv(i)
  values = df.loc[0, :].values.tolist()
  a = (values[7] / values[1]) * 100 #left hand - Lower Left quadrant
  L_LL.append(a)
  b = (values[18] / values[10]) * 100 #right hand - Lower Right quadrant
  R_LR.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_LL, R_LR, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.876618,31,two-sided,0.387437,"[-5.5, 2.19]",0.118833,0.269,0.099883


In [ ]:
#prints means under comparison
print(f'mean_L_LL: {Average(L_LL)}   mean_R_LR: {Average(R_LR)}')

mean_L_LL: 63.80119756556481   mean_R_LR: 65.45502333423659


#Statistical differences between the different hand interactions

In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

R_portable_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/RightHanders/Hands_Portable/*.csv'))

print("Total number of files: " + str(len(R_portable_counts)))

Total number of files: 32


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

R_stationary_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/RightHanders/Hands_Stationary/*.csv'))

print("Total number of files: " + str(len(R_stationary_counts)))

Total number of files: 32


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

R_noContact_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/RightHanders/Hands_NoContact/*.csv'))

print("Total number of files: " + str(len(R_noContact_counts)))

Total number of files: 32


In [ ]:
#Accessing the folder where frame counts by participant are stored
#glob turns all the content of the folder into a list 

import glob

R_self_counts = sorted(glob.glob(r'/content/gdrive/MyDrive/HandVisualField_Paper/hands_csvs_by_participant/RightHanders/Hands_Self/*.csv'))

print("Total number of files: " + str(len(R_self_counts)))

Total number of files: 32


##Stationary objects vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_stat = []
R_low_noC = []

for stat, noC in zip(R_stationary_counts, R_noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - stationary ojects
  R_low_stat.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_stat, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-9.422868,31,two-sided,1.299755e-10,"[-20.78, -13.38]",1.36353,7.854e+07,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_R_lower_stationary: {Average(R_low_stat)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_stationary: 61.99314116806956   mean_R_lower_noContact: 79.07294299761148


##Portable objects vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_port = []
R_low_noC = []

for port, noC in zip(R_portable_counts, R_noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[13] / values_port[11]) * 100 #right hand - lower - portable ojects
  R_low_port.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_port, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.487043,31,two-sided,0.000093,"[-12.76, -4.78]",0.621486,273.75,0.925749


In [ ]:
#prints means under comparison
print(f'mean_R_lower_portable: {Average(R_low_port)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_portable: 70.30143511155723   mean_R_lower_noContact: 79.07294299761148


##Self-contact vs No object interaction - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_noC = []

for selfie, noC in zip(R_self_counts, R_noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - self
  R_low_self.append(a)
  b = (values_noC[13] / values_noC[11]) * 100 #right hand - lower - no contact
  R_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.930211,31,two-sided,0.35945,"[-2.3, 6.17]",0.14226,0.281,0.122087


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_noContact: {Average(R_low_noC)}')

mean_R_lower_self: 81.00562818920417   mean_R_lower_noContact: 79.07294299761148


##Self-contact vs Portable objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_port = []

for selfie, port in zip(R_self_counts, R_portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - stationary ojects
  R_low_self.append(a)
  b = (values_port[13] / values_port[11]) * 100 #right hand - lower - port
  R_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.724411,31,two-sided,0.000047,"[6.08, 15.33]",0.628867,505.558,0.931259


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_portable: {Average(R_low_port)}')

mean_R_lower_self: 81.00562818920417   mean_R_lower_portable: 70.30143511155723


##Self-contact vs Stationary objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_self = []
R_low_stat = []

for selfie, stat in zip(R_self_counts, R_stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[13] / values_self[11]) * 100 #right hand - lower - stationary ojects
  R_low_self.append(a)
  b = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - port
  R_low_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_self, R_low_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,8.453018,31,two-sided,1.508014e-09,"[14.43, 23.6]",1.208674,7.719e+06,1.0


In [ ]:
#prints means under comparison
print(f'mean_R_lower_self: {Average(R_low_self)}   mean_R_lower_stationary: {Average(R_low_stat)}')

mean_R_lower_self: 81.00562818920417   mean_R_lower_stationary: 61.99314116806956


##Stationary vs Portable objects - Lower hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_low_stat = []
R_low_port = []

for stat, port in zip(R_stationary_counts, R_portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[13] / values_stat[11]) * 100 #right hand - lower - stationary ojects
  R_low_stat.append(a)
  b = (values_port[13] / values_port[11]) * 100 #right hand - lower - port
  R_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_low_stat, R_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-3.965757,31,two-sided,0.000402,"[-12.58, -4.04]",0.513226,73.268,0.802978


In [ ]:
#prints means under comparison
print(f'mean_R_lower_stationary: {Average(R_low_stat)}   mean_R_lower_portable: {Average(R_low_port)}')

mean_R_lower_stationary: 61.99314116806956   mean_R_lower_portable: 70.30143511155723


##Stationary objects vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_stat = []
R_up_noC = []

for stat, noC in zip(R_stationary_counts, R_noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - stationary ojects
  R_up_stat.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_stat, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,9.422868,31,two-sided,1.299755e-10,"[13.38, 20.78]",1.36353,7.854e+07,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_R_upper_stationary: {Average(R_up_stat)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_stationary: 38.00685883193045   mean_R_upper_noContact: 20.927057002388548


##Portable objects vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_port = []
R_up_noC = []

for port, noC in zip(R_portable_counts, R_noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[12] / values_port[11]) * 100 #right hand - upper - portable ojects
  R_up_port.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_port, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,4.487043,31,two-sided,0.000093,"[4.78, 12.76]",0.621486,273.75,0.925749


In [ ]:
#prints means under comparison
print(f'mean_R_upper_portable: {Average(R_up_port)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_portable: 29.69856488844277   mean_R_upper_noContact: 20.927057002388548


##Self-contact vs No object interaction - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_noC = []

for selfie, noC in zip(R_self_counts, R_noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - self
  R_up_self.append(a)
  b = (values_noC[12] / values_noC[11]) * 100 #right hand - upper - no contact
  R_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.930211,31,two-sided,0.35945,"[-6.17, 2.3]",0.14226,0.281,0.122087


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_noContact: {Average(R_up_noC)}')

mean_R_upper_self: 18.994371810795833   mean_R_upper_noContact: 20.927057002388548


##Self-contact vs Portable objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_port = []

for selfie, port in zip(R_self_counts, R_portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - stationary ojects
  R_up_self.append(a)
  b = (values_port[12] / values_port[11]) * 100 #right hand - upper - port
  R_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-4.724411,31,two-sided,0.000047,"[-15.33, -6.08]",0.628867,505.558,0.931259


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_portable: {Average(R_up_port)}')

mean_R_upper_self: 18.994371810795833   mean_R_upper_portable: 29.69856488844277


##Self-contact vs Stationary objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_self = []
R_up_stat = []

for selfie, stat in zip(R_self_counts, R_stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[12] / values_self[11]) * 100 #right hand - upper - stationary ojects
  R_up_self.append(a)
  b = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - port
  R_up_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_self, R_up_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-8.453018,31,two-sided,1.508014e-09,"[-23.6, -14.43]",1.208674,7.719e+06,1.0


In [ ]:
#prints means under comparison
print(f'mean_R_upper_self: {Average(R_up_self)}   mean_R_upper_stationary: {Average(R_up_stat)}')

mean_R_upper_self: 18.994371810795833   mean_R_upper_stationary: 38.00685883193045


##Stationary vs Portable objects - Upper hemi scene - Right Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

R_up_stat = []
R_up_port = []

for stat, port in zip(R_stationary_counts, R_portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[12] / values_stat[11]) * 100 #right hand - upper - stationary ojects
  R_up_stat.append(a)
  b = (values_port[12] / values_port[11]) * 100 #right hand - upper - port
  R_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(R_up_stat, R_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.965757,31,two-sided,0.000402,"[4.04, 12.58]",0.513226,73.268,0.802978


In [ ]:
#prints means under comparison
print(f'mean_R_upper_stationary: {Average(R_up_stat)}   mean_R_upper_portable: {Average(R_up_port)}')

mean_R_upper_stationary: 38.00685883193045   mean_R_upper_portable: 29.69856488844277


##Stationary objects vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_stat = []
L_low_noC = []

for stat, noC in zip(R_stationary_counts, R_noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - stationary ojects
  L_low_stat.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_stat, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-9.775954,31,two-sided,5.490412e-11,"[-15.56, -10.19]",1.293367,1.779e+08,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))


In [ ]:
#prints means under comparison
print(f'mean_L_lower_stationary: {Average(L_low_stat)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_stationary: 73.51901789301472   mean_L_lower_noContact: 86.39222434766602


##Portable objects vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_port = []
L_low_noC = []

for port, noC in zip(R_portable_counts, R_noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[4] / values_port[2]) * 100 #left hand - lower - portable ojects
  L_low_port.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_port, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-5.55507,31,two-sided,0.000004,"[-15.79, -7.31]",0.947938,4476.414,0.999381


In [ ]:
#prints means under comparison
print(f'mean_L_lower_portable: {Average(L_low_port)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_portable: 74.84356287328657   mean_L_lower_noContact: 86.39222434766602


##Self-contact vs No object interaction - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_noC = []

for selfie, noC in zip(R_self_counts, R_noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self
  L_low_self.append(a)
  b = (values_noC[4] / values_noC[2]) * 100 #left hand - lower - no contact
  L_low_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-1.762796,31,two-sided,0.087792,"[-8.25, 0.6]",0.315436,0.752,0.408819


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_noContact: {Average(L_low_noC)}')

mean_L_lower_self: 82.56889174801235   mean_L_lower_noContact: 86.39222434766602


##Self-contact vs Portable objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_port = []

for selfie, port in zip(R_self_counts, R_portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self ojects
  L_low_self.append(a)
  b = (values_port[4] / values_port[2]) * 100 #left hand - lower - port
  L_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.716243,31,two-sided,0.000798,"[3.49, 11.97]",0.486338,39.745,0.759628


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_portable: {Average(L_low_port)}')

mean_L_lower_self: 82.56889174801235   mean_L_lower_portable: 74.84356287328657


##Self-contact vs Stationary objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_self = []
L_low_stat = []

for selfie, stat in zip(R_self_counts, R_stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[4] / values_self[2]) * 100 #left hand - lower - self
  L_low_self.append(a)
  b = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - port
  L_low_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_self, L_low_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.570186,31,two-sided,0.001187,"[3.88, 14.22]",0.635228,27.989,0.93575


In [ ]:
#prints means under comparison
print(f'mean_L_lower_self: {Average(L_low_self)}   mean_L_lower_stationary: {Average(L_low_stat)}')

mean_L_lower_self: 82.56889174801235   mean_L_lower_stationary: 73.51901789301472


##Stationary vs Portable objects - Lower hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_low_stat = []
L_low_port = []

for stat, port in zip(R_stationary_counts, R_portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[4] / values_stat[2]) * 100 #left hand - lower - stationary ojects
  L_low_stat.append(a)
  b = (values_port[4] / values_port[2]) * 100 #left hand - lower - port
  L_low_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_low_stat, L_low_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-0.61173,31,two-sided,0.545177,"[-5.74, 3.09]",0.092628,0.225,0.080042


In [ ]:
#prints means under comparison
print(f'mean_L_lower_stationary: {Average(L_low_stat)}   mean_L_lower_portable: {Average(L_low_port)}')

mean_L_lower_stationary: 73.51901789301472   mean_L_lower_portable: 74.84356287328657


##Stationary objects vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_stat = []
L_up_noC = []

for stat, noC in zip(R_stationary_counts, R_noContact_counts):
  df_stat = pd.read_csv(stat)
  df_noC = pd.read_csv(noC)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - stationary ojects
  L_up_stat.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_stat, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,9.775954,31,two-sided,5.490412e-11,"[10.19, 15.56]",1.293367,1.779e+08,1.0


In [ ]:
def Average(lst):
  '''formula to calculate mean from list of values'''
  return(sum(lst) / len(lst))

In [ ]:
#prints means under comparison
print(f'mean_L_upper_stationary: {Average(L_up_stat)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_stationary: 26.48098210698526   mean_L_upper_noContact: 13.607775652333993


##Portable objects vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_port = []
L_up_noC = []

for port, noC in zip(R_portable_counts, R_noContact_counts):
  df_port = pd.read_csv(port)
  df_noC = pd.read_csv(noC)

  values_port = df_port.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_port[3] / values_port[2]) * 100 #left hand - upper - portable ojects
  L_up_port.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_port, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,5.55507,31,two-sided,0.000004,"[7.31, 15.79]",0.947938,4476.414,0.999381


In [ ]:
#prints means under comparison
print(f'mean_L_upper_portable: {Average(L_up_port)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_portable: 25.156437126713428   mean_L_upper_noContact: 13.607775652333993


##Self-contact vs No object interaction - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_noC = []

for selfie, noC in zip(R_self_counts, R_noContact_counts):
  df_self = pd.read_csv(selfie)
  df_noC = pd.read_csv(noC)

  values_self = df_self.loc[0, :].values.tolist()
  values_noC = df_noC.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self
  L_up_self.append(a)
  b = (values_noC[3] / values_noC[2]) * 100 #left hand - upper - no contact
  L_up_noC.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_noC, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.762796,31,two-sided,0.087792,"[-0.6, 8.25]",0.315436,0.752,0.408819


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_noContact: {Average(L_up_noC)}')

mean_L_upper_self: 17.43110825198766   mean_L_upper_noContact: 13.607775652333993


##Self-contact vs Portable objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_port = []

for selfie, port in zip(R_self_counts, R_portable_counts):
  df_self = pd.read_csv(selfie)
  df_port = pd.read_csv(port)

  values_self = df_self.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self ojects
  L_up_self.append(a)
  b = (values_port[3] / values_port[2]) * 100 #left hand - upper - port
  L_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-3.716243,31,two-sided,0.000798,"[-11.97, -3.49]",0.486338,39.745,0.759628


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_portable: {Average(L_up_port)}')

mean_L_upper_self: 17.43110825198766   mean_L_upper_portable: 25.156437126713428


##Self-contact vs Stationary objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_self = []
L_up_stat = []

for selfie, stat in zip(R_self_counts, R_stationary_counts):
  df_self = pd.read_csv(selfie)
  df_stat = pd.read_csv(stat)

  values_self = df_self.loc[0, :].values.tolist()
  values_stat = df_stat.loc[0, :].values.tolist()
  
  a = (values_self[3] / values_self[2]) * 100 #left hand - upper - self
  L_up_self.append(a)
  b = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - port
  L_up_stat.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_self, L_up_stat, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,-3.570186,31,two-sided,0.001187,"[-14.22, -3.88]",0.635228,27.989,0.93575


In [ ]:
#prints means under comparison
print(f'mean_L_upper_self: {Average(L_up_self)}   mean_L_upper_stationary: {Average(L_up_stat)}')

mean_L_upper_self: 17.43110825198766   mean_L_upper_stationary: 26.48098210698526


##Stationary vs Portable objects - Upper hemi scene - Left Hand

In [ ]:
#For loop, goes through counts and pulls relavant ones
#Calculates proportions and adds them to their respective list

import pandas as pd

L_up_stat = []
L_up_port = []

for stat, port in zip(R_stationary_counts, R_portable_counts):
  df_stat = pd.read_csv(stat)
  df_port = pd.read_csv(port)

  values_stat = df_stat.loc[0, :].values.tolist()
  values_port = df_port.loc[0, :].values.tolist()
  
  a = (values_stat[3] / values_stat[2]) * 100 #left hand - upper - stationary ojects
  L_up_stat.append(a)
  b = (values_port[3] / values_port[2]) * 100 #left hand - upper - port
  L_up_port.append(b)

In [ ]:
import pingouin as pt

# Python paired sample t-test:
pt.ttest(L_up_stat, L_up_port, paired=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,0.61173,31,two-sided,0.545177,"[-3.09, 5.74]",0.092628,0.225,0.080042


In [ ]:
#prints means under comparison
print(f'mean_L_upper_stationary: {Average(L_up_stat)}   mean_L_upper_portable: {Average(L_up_port)}')

mean_L_upper_stationary: 26.48098210698526   mean_L_upper_portable: 25.156437126713428
